# 식품의약품안전처_수입식품 해외제조업소 정보  (식의약헬스종합정보)
---
- 작성자 : 안지은
- 실행일 : 2023-01-10

In [1]:
import requests
import pprint
import json
import pandas as pd
from datetime import datetime,  timedelta
import tqdm 
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

key = '7bBeVcu1HiKEYVutnwGstfISHvCJKzXcizQ2iFZlhBCaQobHPqOod8G%2BcwO05R81TptPWRtVcKUSqmhqOqCGXQ%3D%3D'

In [2]:
# 총 데이터 수 구하는 함수
def get_total_cnt(key):
    url = 'https://apis.data.go.kr/1471000/IprtFoodOvseaMnftBsshInfoService01/getIprtFoodOvseaMnftBsshInfoInq01?serviceKey=%s&type=json'%(key)
    response = requests.get(url, verify=False)
    contents = response.text
    json_ob = json.loads(contents)
    cnt = int(json_ob['body']['totalCount'])
    return cnt

In [3]:
# api 컨텐츠(json - data) 받아오는 함수
def get_api_conts(key,page):
    
    url = 'https://apis.data.go.kr/1471000/IprtFoodOvseaMnftBsshInfoService01/getIprtFoodOvseaMnftBsshInfoInq01?serviceKey=%s&pageNo=%s&numOfRows=100&type=json'%(key,str(page))
    
    response = requests.get(url, verify=False)
    contents = response.text
    json_ob = json.loads(contents)
    data = json_ob['body']['items']
    
    return data

### API 가져오기
---
- 한 페이지 당  100개 씩 제한
- 총 96792개 행
- body - item 리스트 안에 다시 딕셔너리로 item이 있음

In [4]:
foreign_food = []
cnt = get_total_cnt(key)
page = int(cnt/100) + 1
for i in tqdm(range(1,page+1)):
    js = get_api_conts(key,i)
    for item in js:
        foreign_food.append(item['item'])


100%|██████████| 968/968 [11:43<00:00,  1.38it/s]


### 데이터 프레임 변환
---
- 컬럼명 : API 목록

In [5]:
foreign_food_df = pd.json_normalize(foreign_food)
foreign_food_df.columns = ['해외제조업소코드','해외제조업소명','해외제조업소영업구분코드','해외제조업소영업구분명',
'식품구분코드','식품구분명','시설취소일시','국가코드','국가명','지역코드','지역명','식품안전관리시스템인증여부',
'인증명','인증기관명','인증기관일시','인증기관만료일시',
'취하여부','취하일시','취소중단코드','취소중단명','수동등록구분코드',
'식품유통시작일시','식품유통종료일시','수산시작일시','수입중단번호']
foreign_food_df

,해외제조업소코드,해외제조업소명,해외제조업소영업구분코드,해외제조업소영업구분명,식품구분코드,식품구분명,시설취소일시,국가코드,국가명,지역코드,...,인증기관만료일시,취하여부,취하일시,취소중단코드,취소중단명,수동등록구분코드,식품유통시작일시,식품유통종료일시,수산시작일시,수입중단번호
0,AE000000002,AHMAD TEA FZ LLC,1,식품 또는 식품첨가물 제조.가공,2,가공식품,None,AE,아랍에미리트 연합,AE-RK,...,2024-03-13 00:00:00,N,None,N,등록,N,2016-06-09 00:00:00,2024-06-08 00:00:00,None,None
1,AE000000003,SEVILLE PRODUCTS LIMITED,1,식품 또는 식품첨가물 제조.가공,2,가공식품,None,AE,아랍에미리트 연합,AE-DU,...,2023-12-29 00:00:00,N,None,N,등록,N,2022-06-13 00:00:00,2024-06-12 00:00:00,None,None
2,AE000000006,ARAB BEVERAGES EST,1,식품 또는 식품첨가물 제조.가공,"2,5","가공식품,식품첨가물",None,AE,아랍에미리트 연합,AE-DU,...,None,N,None,N,등록,N,2020-08-28 00:00:00,2022-08-27 00:00:00,None,None
3,AE000000007,AL KHALEEJ SUGAR CO. (L.L.C.),1,식품 또는 식품첨가물 제조.가공,5,식품첨가물,None,AE,아랍에미리트 연합,AE-DU,...,2023-03-30 00:00:00,N,None,N,등록,N,2020-06-02 00:00:00,2022-06-01 00:00:00,None,None
4,AE000000008,HUNTERS FOODS LTD. FZCO,1,식품 또는 식품첨가물 제조.가공,"2,5","가공식품,식품첨가물",None,AE,아랍에미리트 연합,AE-DU,...,2018-10-01 00:00:00,N,None,N,등록,N,2020-08-12 00:00:00,2022-08-11 00:00:00,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96787,ZM000000001,NORTHERN COFFEE CORPORATION LIMITED.,3,농산물 포장,1,농산물,None,ZM,잠비아,ZM-05,...,None,N,None,N,등록,N,2021-02-23 00:00:00,2023-02-22 00:00:00,None,None
96788,ZM000000002,ADCOMS INVESTEMENRS LTD,1,식품 또는 식품첨가물 제조.가공,2,가공식품,None,ZM,잠비아,ZM-09,...,2020-01-24 00:00:00,N,None,N,등록,N,2019-10-24 00:00:00,2021-10-23 00:00:00,None,None
96789,ZM000000003,ZAMBIA COFFEE GROWERS ASSOCIATION LTD,3,농산물 포장,"1,2","농산물,가공식품",None,ZM,잠비아,ZM-09,...,None,N,None,N,등록,N,2020-03-30 00:00:00,2022-03-29 00:00:00,None,None
96790,ZM000000004,MORINGA INITIATIVE LTD,1,식품 또는 식품첨가물 제조.가공,2,가공식품,None,ZM,잠비아,ZM-10,...,None,N,None,N,등록,N,2019-12-06 00:00:00,2021-12-05 00:00:00,None,None
